In [1]:
import pandas as pd

df = pd.read_excel('/Users/martinwolf/Python/Superdepot Reporting/Data/SKU_Bin.xlsx')
df.head()

,Zeilenbeschriftungen,Gesamtergebnis,Median CS,Pal Menge,Bins,Unnamed: 5
0,10138900,0.0,NaN,21.0,0.0,NaN
1,10139126,0.0,NaN,42.0,0.0,NaN
2,75011029,0.0,NaN,1.0,0.0,NaN
3,10129113,0.0,NaN,21.0,0.0,NaN
4,75009824,0.0,NaN,1.0,0.0,NaN


In [8]:
df = df.sample(frac=1).reset_index(drop=True)
df
df.to_excel('/Users/martinwolf/Python/Superdepot Reporting/Data/SKU_BinNeu.xlsx',index=False)